<a href="https://colab.research.google.com/github/ARS-SWRC/GEE-KGCC/blob/main/geekgcc_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This Python notebook tests the functionality of GEE-KGCC and can be run in Google Colab. The WorldClim dataset is used as input, and the produced output is KGCC classifications for the northern hemisphere.**

In [ ]:
!git clone https://github.com/ARS-SWRC/GEE-KGCC
!pip install GEE-KGCC/geekgcc_package

In [ ]:
import ee

#This will open a web browser for log-in steps.
ee.Authenticate()
geeusername = 'yourusername' #Enter your GEE user name.
geeproject = 'ee-yourusername' #Enter your GEE project name.
ee.Initialize(project=geeproject)
#The user must have an existing project.
#Default project names are in the format: "ee-yourusername".
#The web browser log-in steps assist with creating a project
#or one may be created at https://code.earthengine.google.com/

In [ ]:
#ee should be initialized before importing geekgcc.
import geekgcc

dl = False #Download to Google Drive True/False

###############
#PRE-PROCESSING
###############
worldclim_path = 'WORLDCLIM/V1/MONTHLY'
ic = ee.ImageCollection(worldclim_path)

#Northern hemisphere
custom_geo = ee.Geometry.BBox(-180.00, 0.00, 179.99, 89.99)
#Southern hemisphere
#custom_geo = ee.Geometry.BBox(-180.00, 0.00, 179.99, -89.99)

def clip_fn(im_obj):
  im = ee.Image(im_obj)
  clip_im = im.clip(custom_geo)
  return clip_im

ic_clip = ee.ImageCollection(ic.map(clip_fn))
t_scaled_ic = ee.ImageCollection(ic_clip.select('tavg'))

def unit_scaling_fn(im_obj):
  scaled_im = ee.Image(im_obj)
  im = scaled_im.multiply(0.1)
  return im

#####################
#USER-SUPPLIED INPUTS
#####################

#Monthly mean precipitation (12 images)
p_ic = ee.ImageCollection(ic_clip.select('prec'))
#Monthly mean temperature (12 images)
t_ic = ee.ImageCollection(t_scaled_ic.map(unit_scaling_fn))

###############
#CLASSIFICATION
###############
type_im = geekgcc.KGCC.classify(p_ic, t_ic, 'north')

############
#DOWNLOADING
############
if dl == True:
  #Scale is a client-side object that can be determined from
  #the first image in the collection used to calculate p_ic and t_ic.
  scale = ic.first().projection().nominalScale().getInfo()
  #client-side object of the downloaded geotif file name.
  file_name = 'test_kgcc_map'
  geekgcc.KGCC.download(type_im, custom_geo, scale, file_name)

############
#CLASS INDEX
############
#This is useful for looking up the class name.
class_dict = geekgcc.KGCC.get_class_index()
print('Lookup index for getting KGGC type from pixel value:')
print(class_dict)

In [ ]:
############
#VISUALIZING
############

#Run this block to load an html map widget.
#Should be run in own block (as currently done).
#In Colab and other online notebook editors, use "Map"
#to load the map widget.
#On a desktop code editor or an editor without
#an IPython Kernel use "Map.save()".

import geemap
geemap.ee_initialize()

Map = geemap.Map(center=[40,-100], zoom=3)
vis_params = geekgcc.KGCC.get_vis_params()
Map.addLayer(type_im, vis_params)

#Use "Map.save()" when running on a desktop code editor or outside of an
#IPython Kernel.
#This will save the map widget as an html doc in the working directory
#Map.save('map.html')

#Use "Map" when using an online web-based editor like Google Colab.
Map